# Create sample Q/A dataset from knowledge base stored in vector DB 

In [1]:
import ollama
from qdrant_client import QdrantClient
import json

/opt/anaconda3/envs/rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Read parameters file

parameters_file = "params.json"

with open(parameters_file, 'r') as fh:
    params = json.loads(fh.read())

In [2]:
# Prompt to generate Question-Answer pairs from the provided text

prompt = """
Write a FAQ page for a college called TBC College which is based in Toronto, Ontario. 

Write 2 questions and answers from the provided context. 
Organize the questions and answers in a python list with question and answer stored in a dictionary, and use clear and concise language.
Use the folowing format for dictionary,
{"question":,
"answer":}

Only provide the list as output. Do not output any other text.

Context:

"""

In [3]:
# Create connection with Qdrant DB
client = QdrantClient("localhost", port=6333)

In [5]:
# Fetch records from the collection

data = client.scroll(
    collection_name=params["collection_name"],
    with_payload=True,
    with_vectors=False,
    limit=100
)[0]

In [6]:
len(data)

100

In [7]:
# Collect the text from records in a list
records = [x.payload['text'] for x in data]

In [8]:
from tqdm.auto import tqdm

In [9]:
# Create questions and answers for the provided data using Language Model

qa = []

for record in tqdm(records):
    result = ollama.generate(model=params["lm_name"], prompt=prompt+record)
    
    try:
        response = json.loads(result['response'])
        clean_response = [x for x in response if not(x['question']=='' or x['answer']=='')]
        qa.extend(clean_response)
    except:
        pass
    

  1%|          | 1/100 [00:06<11:02,  6.70s/it]

{'model': 'mistral', 'created_at': '2024-07-26T15:12:35.261957Z', 'response': ' [\n  {\n    "question": "What is Student Ambassador Program at TBC College?",\n    "answer": "The Student Ambassador Program at TBC College provides friendly and respectful peer support for students, utilizing the lived experience of international students to help build a healthy and successful college experience."\n  },\n  {\n    "question": "How can I access more information about TBC College\'s Student Ambassador Program?",\n    "answer": "You can find more information by clicking on this link: https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg"\n  }\n]', 'done': True, 'done_reason': 'stop', 'context': [3, 1027, 781, 6006, 1032, 14121, 29592, 3652, 1122, 1032, 7070, 2755, 1088, 5506, 6904, 1458, 1117, 3586, 1065, 15752, 29493, 19340, 29491, 29473, 781, 781, 6006, 29473, 29518, 4992, 1072, 11962, 1245, 1040, 4625, 3526, 29491, 29473, 781, 22598, 142

  2%|▏         | 2/100 [00:14<11:46,  7.21s/it]

{'model': 'mistral', 'created_at': '2024-07-26T15:12:42.83408Z', 'response': ' [\n  {\n    "question": "What is TBC College\'s approach towards student support?",\n    "answer": "TBC College provides peer support, which recognizes that students naturally turn to each other for support and connection. Student Ambassadors utilize their lived experience as international students to offer friendly, respectful support, helping students build a healthy and successful college experience."\n  },\n  {\n    "question": "How can I access more information about TBC College\'s student support program?",\n    "answer": "You can click here: https://tbcollege0.sharepoint.com/:b:/s/ExternShare/EWPHCe-qNINHgqTW486g3-YBD-0vkdNy2Ke0sKixC01BLQ?e=cAGXGg for more details."\n  }\n]', 'done': True, 'done_reason': 'stop', 'context': [3, 1027, 781, 6006, 1032, 14121, 29592, 3652, 1122, 1032, 7070, 2755, 1088, 5506, 6904, 1458, 1117, 3586, 1065, 15752, 29493, 19340, 29491, 29473, 781, 781, 6006, 29473, 29518, 499

  3%|▎         | 3/100 [00:21<11:57,  7.39s/it]

...

{'model': 'mistral', 'created_at': '2024-07-26T15:24:20.24254Z', 'response': ' [\n  {"question": "What is the seating option for international travel at TBC College?",\n   "answer": "Economy class seating is the standard option for international travel."},\n  {\n    "question": "Under what conditions can Business class be permitted for international flights at TBC College?",\n    "answer": "Business class may be permitted on international flights with prior approval of the President if the one-way flight time exceeds 9 hours (not including layovers)."\n  },\n  {\n    "question": "What is the standard accommodation for students at TBC College?",\n    "answer": "The standard for accommodation is a single room in a business class hotel."\n  }\n]', 'done': True, 'done_reason': 'stop', 'context': [3, 1027, 781, 6006, 1032, 14121, 29592, 3652, 1122, 1032, 7070, 2755, 1088, 5506, 6904, 1458, 1117, 3586, 1065, 15752, 29493, 19340, 29491, 29473, 781, 781, 6006, 29473, 29518, 4992, 1072, 11962, 

 99%|█████████▉| 99/100 [11:56<00:06,  6.34s/it]

{'model': 'mistral', 'created_at': '2024-07-26T15:24:24.988521Z', 'response': ' [\n  {"question": "What is the policy for accommodation expenses for employees at TBC College?",\n   "answer": "If an employee chooses to stay with relatives or friends, they may claim $30 per night in lieu of accommodation costs. No receipt is required."},\n  {"question": "What is the policy for meal reimbursements at TBC College?",\n   "answer": "Meals purchased during travel will be reimbursed to a maximum amount per meal. All claims for meals must be accompanied by an itemized receipt."}\n]', 'done': True, 'done_reason': 'stop', 'context': [3, 1027, 781, 6006, 1032, 14121, 29592, 3652, 1122, 1032, 7070, 2755, 1088, 5506, 6904, 1458, 1117, 3586, 1065, 15752, 29493, 19340, 29491, 29473, 781, 781, 6006, 29473, 29518, 4992, 1072, 11962, 1245, 1040, 4625, 3526, 29491, 29473, 781, 22598, 1421, 1040, 4992, 1072, 11962, 1065, 1032, 22734, 2042, 1163, 3764, 1072, 5140, 11909, 1065, 1032, 20211, 29493, 1072, 1706

100%|██████████| 100/100 [12:02<00:00,  7.22s/it]

{'model': 'mistral', 'created_at': '2024-07-26T15:24:30.928878Z', 'response': ' [\n{"question": "Can I claim meals at TBC College if they are included in my transportation fare or registration fees?",\n"answer": "No, meals cannot be claimed when they are included in the transportation fare or the cost of registration fees."},\n{"question": "What expenses can be claimed during a business trip at TBC College?",\n"answer": "At TBC College, you can claim costs for parking and tolls, bus and taxi fares, reasonable personal calls from home each night away on business trips, additional business expenses, and cash advances (only if not using a corporate card). Receipts must accompany the expense report for all claims."}\n]', 'done': True, 'done_reason': 'stop', 'context': [3, 1027, 781, 6006, 1032, 14121, 29592, 3652, 1122, 1032, 7070, 2755, 1088, 5506, 6904, 1458, 1117, 3586, 1065, 15752, 29493, 19340, 29491, 29473, 781, 781, 6006, 29473, 29518, 4992, 1072, 11962, 1245, 1040, 4625, 3526, 2949

In [10]:
# Save QA list in a json file
with open(params["qa_filename"], 'w') as fh:
    fh.write(json.dumps(qa))